In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import glob
import numpy as np
import pandas as pd

from model import DeepBrain
from dataset import BrainDataset

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/neurohyper/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import torch
from torch import nn
from torch.functional import F
torch.backends.cudnn.benchmark = False
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

In [4]:
file_list = ['./119833_Motor_1_lh.nii.gz']
label_list = [[0,0,0,1,0,0,0]]

dataset = BrainDataset(file_list, label_list, is_train=False)

In [5]:
model = DeepBrain().to(device)
model.load_state_dict(torch.load('./checkpoint/checkpoint_24.pth.tar')['state_dict'])

model.py:87: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))


In [6]:
_ = model.eval()
with torch.no_grad():
    inputs, label = dataset[0]
    pred_prob = F.softmax(model(torch.FloatTensor(inputs).unsqueeze(0).to(device)))

dataset.py:26: RuntimeWarning: divide by zero encountered in divide
  data = data / data.max(axis=3)[:, :, :, np.newaxis]
dataset.py:26: RuntimeWarning: invalid value encountered in divide
  data = data / data.max(axis=3)[:, :, :, np.newaxis]
/home/neurohyper/anaconda2/lib/python2.7/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/home/neurohyper/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


In [7]:
pred_prob

tensor([[6.1537e-09, 1.0000e+00, 5.6260e-11, 2.5552e-15, 6.2870e-09, 4.2733e-11,
         6.1202e-08]], device='cuda:0')

# Visualization

In [8]:
import nibabel as nib
from guided_backprop import GuidedBackprop

In [9]:
_ = model.eval()

In [10]:
# Remove LogSoftmax
model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])

In [11]:
GBP = GuidedBackprop(model)

In [12]:
inputs, label = dataset[0]
input_img = nn.Parameter(torch.FloatTensor(inputs).unsqueeze(0), requires_grad=True).to(device)

In [14]:
guided_grads = GBP.generate_gradients(input_img, label)

In [15]:
export_gradient = np.zeros((27, 91, 109, 91))
export_gradient[:, 8:-8, 8:-8, :-10] = guided_grads
nifti_img = nib.Nifti1Image(export_gradient.transpose(1, 2, 3, 0), np.eye(4))
nifti_img.to_filename('vis_tmp.nii.gz')